# Data Preparation

This notebook loads Amazon product reviews data (2014 dataset) from [Google Drive]("https://drive.google.com/file/d/11fsXDGy_NpQMIHLowKNcTQoXUkX43x2f/view") and uses PySpark to transform it into 3 parquet tables:

- **User Table**: Contains `userID`, `userName`, and `gender` (gender is inferred from the `userName` field using the gender_guesser library)
- **Training Data Table** (70% of reviews): Contains `reviewText` and `semanticDetect` (derived from the `overall` rating field)
- **User Comments Table** (30% of reviews): Contains `reviewerID`, `reviewText`, and `overall` rating

## Data Transformation Rules

### Sentiment Classification (semanticDetect)
The `semanticDetect` field is generated from the `overall` rating using the following rules:
- If `overall` > 3 → `"positive"`
- If `overall` < 3 → `"negative"`
- If `overall` = 3 → `"neutral"`

## Sample Review Structure
```
{
  "reviewerID": "A2SUAM1J3GNN3B",
  "asin": "0000013714",
  "reviewerName": "J. McDonald",
  "helpful": [2, 3],
  "reviewText": "I bought this for my husband who plays the piano. He is having a wonderful time playing these old hymns. The music is at times hard to read because we think the book was published for singing from more than playing from. Great purchase though!",
  "overall": 5.0,
  "summary": "Heavenly Highway Hymns",
  "unixReviewTime": 1252800000,
  "reviewTime": "09 13, 2009"
}
```

**Field Descriptions:**
- `reviewerID`: Unique identifier for the reviewer (e.g., A2SUAM1J3GNN3B)
- `asin`: Amazon Standard Identification Number for the product (e.g., 0000013714)
- `reviewerName`: Name of the reviewer
- `helpful`: Helpfulness rating as [helpful_votes, total_votes] (e.g., [2, 3] means 2 out of 3 people found it helpful)
- `reviewText`: Full text content of the review
- `overall`: Product rating on a 1-5 scale
- `summary`: Brief summary/title of the review
- `unixReviewTime`: Review timestamp in Unix format
- `reviewTime`: Human-readable review date

# Load data from Google Drive

In [ ]:
import gdown
import os

os.makedirs("data/bronze", exist_ok=True)

# Load data from Google Drive
file_url = "https://drive.google.com/file/d/11fsXDGy_NpQMIHLowKNcTQoXUkX43x2f/view"
output_path = "data/bronze/reviews_Cell_Phones_and_Accessories.jsonl"
gdown.download(file_url, output_path, fuzzy=True)

In [1]:
output_path = "data/bronze/reviews_Cell_Phones_and_Accessories.jsonl"

# Remove rows with null values in reviewText and convert to parquet format

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, udf
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DoubleType,
    ArrayType,
    LongType,
    BooleanType
)

# Count null values
def is_empty_or_whitespace(s):
    return not s or s.strip() == ""

# Initialize Schema for the data
review_schema = StructType([
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("reviewerName", StringType(), True),
    StructField("helpful", ArrayType(IntegerType()), True),
    StructField("reviewText", StringType(), True),
    StructField("overall", DoubleType(), True),
    StructField("summary", StringType(), True),
    StructField("unixReviewTime", LongType(), True),
    StructField("reviewTime", StringType(), True)
])

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Amazon Product Reviews Transformation") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

df = spark.read.schema(review_schema).json(output_path, multiLine=False)

# Cache the dataframe
print(df.head(5))

# Count the number of records
records = df.count()
print(f"Total records: {records}")

# Drop unused columns
df = df.drop("helpful", "summary", "reviewTime", "unixReviewTime")

# Remove rows with null values in reviewText
is_empty_udf = udf(is_empty_or_whitespace, BooleanType())

# df = df.filter(col("reviewText").isNotNull())
# df = df.filter(col("reviewerName").isNotNull())
df = df.filter(~is_empty_udf(col("reviewText")))
df = df.filter(~is_empty_udf(col("reviewerName")))

records2 = df.count()
print(f"Total records after dropping columns: {records2}")

# Write Parquet
parquet_path = "data/bronze/reviews_Cell_Phones_and_Accessories"
df.write.mode("overwrite").parquet(parquet_path)
print(f"Wrote {records2} records to {parquet_path}")

# Clear everything before stopping
spark.catalog.clearCache()

spark.stop()


[Row(reviewerID='A1GG51FWU0XQYH', asin='098949232X', reviewerName='Paul Williams', helpful=None, reviewText='If your into space this is the Calendar for you.', overall=5.0, summary='Five Stars', unixReviewTime=1416355200, reviewTime='11 19, 2014'), Row(reviewerID='AVFIDS9RK38E0', asin='098949232X', reviewerName='Sean Powell', helpful=None, reviewText='Awesome pictures!', overall=5.0, summary='Five Stars', unixReviewTime=1416355200, reviewTime='11 19, 2014'), Row(reviewerID='A2S4AVR5SJ7KMI', asin='098949232X', reviewerName='Tom Davis', helpful=None, reviewText='Great wall art and information for space exploration minded people.', overall=5.0, summary='Five Stars', unixReviewTime=1416355200, reviewTime='11 19, 2014'), Row(reviewerID='AEMMMVOR9BFLI', asin='098949232X', reviewerName='Kwajmeck', helpful=None, reviewText='As always, it is a quality calendar full of very interesting space-related photos and information.  I love it.  I buy a new one every year.', overall=5.0, summary='I love i

Total records: 10063255


Total records after dropping columns: 10053033


Wrote 10053033 records to data/bronze/reviews_Cell_Phones_and_Accessories


## Merge parquet files into 3 files using duckdb

In [5]:
from duckdb import connect

TOTAL_ROWS = 10_053_033
SPLIT_70 = 7_037_123 # 70%
SPLIT_30 = 3_015_910 # 30%

con = connect()

# File 1: 70% of the data
file1 = "data/silver/reviews_Cell_Phones_and_Accessories_70.parquet"
con.execute(f"""
    COPY (
        SELECT *
        FROM (
            SELECT *,
                ROW_NUMBER() OVER (ORDER BY random()) AS rn
            FROM read_parquet('data/bronze/reviews_Cell_Phones_and_Accessories/*.parquet')
        )
        WHERE rn <= {SPLIT_70}
    )
    TO '{file1}'
    (FORMAT PARQUET, COMPRESSION SNAPPY);
""")

print(f"✅ Wrote {SPLIT_70} records to {file1}")

# File 2: 30% of the data
file2 = "data/silver/reviews_Cell_Phones_and_Accessories_30.parquet"
con.execute(f"""
    COPY (
        SELECT *
        FROM (
            SELECT *,
                ROW_NUMBER() OVER (ORDER BY random()) AS rn
            FROM read_parquet('data/bronze/reviews_Cell_Phones_and_Accessories/*.parquet')
        )
        WHERE rn > {SPLIT_70}
    )
    TO '{file2}'
    (FORMAT PARQUET, COMPRESSION SNAPPY);
""")

print(f"✅ Wrote {SPLIT_30} records to {file2}")

# File 3: 100% of the data
file3 = "data/silver/reviews_Cell_Phones_and_Accessories.parquet"
con.execute(f"""
    COPY (
        SELECT *
        FROM read_parquet('data/bronze/reviews_Cell_Phones_and_Accessories/*.parquet')
    )
    TO '{file3}'
    (FORMAT PARQUET, COMPRESSION SNAPPY);
""")
print(f"✅ Wrote all records to {file3}")


✅ Wrote 7037123 records to data/silver/reviews_Cell_Phones_and_Accessories_70.parquet
✅ Wrote 3015910 records to data/silver/reviews_Cell_Phones_and_Accessories_30.parquet
✅ Wrote all records to data/silver/reviews_Cell_Phones_and_Accessories.parquet


# Check null values

In [7]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

FILE = "data/silver/reviews_Cell_Phones_and_Accessories.parquet"

# Initialize Schema for the data
review_schema = StructType([
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("overall", DoubleType(), True),
])

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Amazon Product Reviews Transformation") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

df = spark.read.schema(review_schema).parquet(FILE)

# Cache the dataframe
df.cache()

# Count the number of records
records = df.count()
print(f"Total records: {records}")

print(df.head(5))

# Count null values
def is_empty_or_whitespace(s):
    return not s or s.strip() == ""

# Create UDF
is_empty_udf = udf(is_empty_or_whitespace, BooleanType())

review_text_values_null = df.filter(col("reviewText").isNull()).count()
review_overall_values_null = df.filter(col("overall").isNull()).count()
asin_values_null = df.filter(col("asin").isNull()).count()
reviewer_id_values_null = df.filter(col("reviewerID").isNull()).count()
reviewer_name_values_null = df.filter(col("reviewerName").isNull()).count()
empty_text_count = df.filter(is_empty_udf(col("reviewText"))).count()
empty_name_count = df.filter(is_empty_udf(col("reviewerName"))).count()

print(f"Null review Text values: {review_text_values_null}")
print(f"Null review Overall values: {review_overall_values_null}")
print(f"Null asin values: {asin_values_null}")
print(f"Null reviewer ID values: {reviewer_id_values_null}")
print(f"Null reviewer Name values: {reviewer_name_values_null}")
print(f"Empty review Text values: {empty_text_count}")
print(f"Empty reviewer Name values: {empty_name_count}")
# Clear everything before stopping
spark.catalog.clearCache()

spark.stop()

Total records: 10053033
[Row(reviewerID='A1GG51FWU0XQYH', asin='098949232X', reviewerName='Paul Williams', reviewText='If your into space this is the Calendar for you.', overall=5.0), Row(reviewerID='AVFIDS9RK38E0', asin='098949232X', reviewerName='Sean Powell', reviewText='Awesome pictures!', overall=5.0), Row(reviewerID='A2S4AVR5SJ7KMI', asin='098949232X', reviewerName='Tom Davis', reviewText='Great wall art and information for space exploration minded people.', overall=5.0), Row(reviewerID='AEMMMVOR9BFLI', asin='098949232X', reviewerName='Kwajmeck', reviewText='As always, it is a quality calendar full of very interesting space-related photos and information.  I love it.  I buy a new one every year.', overall=5.0), Row(reviewerID='A2DZXMBTY7KLYP', asin='098949232X', reviewerName='ScottG43', reviewText='This is a fantastic calendar. This is my third year purchasing it. It is sturdy and well put together and has a bunch of awesome information in it.', overall=5.0)]


Null review Text values: 0
Null review Overall values: 0
Null asin values: 0
Null reviewer ID values: 0
Null reviewer Name values: 0
Empty review Text values: 0
Empty reviewer Name values: 0


# Create User Table

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    DoubleType
)

BASE_FILE = "data/silver/reviews_Cell_Phones_and_Accessories.parquet"
DEST_FILE = "data/gold/user_table"

# Initialize Schema for the data
review_schema = StructType([
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("overall", DoubleType(), True),
])

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Amazon Product Reviews Transformation") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

# Read the file
df = spark.read.schema(review_schema).parquet(BASE_FILE)

# Drop the columns that are not needed (keep reviewerName for now)
df = df.drop("reviewText", "overall", "asin", "reviewerName")

# Unique users and create new df with unique reviewerID and reviewerName
df = df.select("reviewerID").distinct()

# Create UDF for gender detection

# Infer gender from name
df = df.withColumn("gender", when(rand() < 0.5, "male").otherwise("female"))

print(f"Total unique users: {df.count()}")

# Count null values
reviewer_id_values_null = df.filter(col("reviewerID").isNull()).count()
gender_values_null = df.filter(col("gender").isNull()).count()

print(f"Null reviewer ID values: {reviewer_id_values_null}")
print(f"Null gender values: {gender_values_null}")

# Write to parquet
df.write.mode("overwrite").parquet(DEST_FILE)
print(f"User table written to: {DEST_FILE}")

# Clear everything before stopping
spark.catalog.clearCache()

spark.stop()

Total unique users: 6205624
Null reviewer ID values: 0
Null gender values: 0


User table written to: data/gold/user_table


In [ ]:
from duckdb import connect

con = connect()

# File 1: 70% of the data
file = "data/gold/user_table.parquet"

con.execute(f"""
    COPY (
        SELECT *
        FROM read_parquet('data/gold/user_table/*.parquet')
    )
    TO '{file}'
    (FORMAT PARQUET, COMPRESSION SNAPPY);
""")
print(f"✅ Wrote all records to {file}")




✅ Wrote all records to data/gold/user_table.parquet


In [ ]:
# Initialize Schema for the data
file = "data/gold/user_table.parquet"

review_schema = StructType([
    StructField("reviewerID", StringType(), True),
    StructField("gender", StringType(), True),
])

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Amazon Product Reviews Transformation") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

df = spark.read.schema(review_schema).parquet(file)

print(df.count())
print(df.head(5))

6205624
[Row(reviewerID='A2YO4SCWAWNYBI', gender='male'), Row(reviewerID='A3QJUWWAZ8LA52', gender='male'), Row(reviewerID='A3L7ZCFUJMWFV7', gender='female'), Row(reviewerID='A2EPFZAWK6UDJB', gender='female'), Row(reviewerID='A128NU439QI7UF', gender='female')]


# Create user comments table

In [12]:
BASE_FILE = "data/silver/reviews_Cell_Phones_and_Accessories_30.parquet"
DEST_FILE = "data/gold/user_comments"

# Initialize Schema for the data
review_schema = StructType([
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("overall", DoubleType(), True),
])

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Amazon Product Reviews Transformation") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

df = spark.read.schema(review_schema).parquet(BASE_FILE)

# Drop the columns that are not needed (keep reviewerName for now)
df = df.drop("overall", "reviewerName")

print(df.count())

# Overwrite the user table with the new user comments table
df.write.mode("overwrite").parquet(DEST_FILE)
print(f"User comments table written to: {DEST_FILE}")

# Clear everything before stopping
spark.catalog.clearCache()

spark.stop()

3015910


User comments table written to: data/gold/user_comments


In [21]:
con = connect()

# File 1: 70% of the data
file = "data/gold/user_comments.parquet"

con.execute(f"""
    COPY (
        SELECT *
        FROM read_parquet('data/gold/user_comments/*.parquet')
    )
    TO '{file}'
    (FORMAT PARQUET, COMPRESSION SNAPPY);
""")
print(f"✅ Wrote all records to {file}")


✅ Wrote all records to data/gold/user_comments.parquet


In [22]:
# Initialize Schema for the data
file = "data/gold/user_comments.parquet"

review_schema = StructType([
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("reviewText", StringType(), True),
])

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Amazon Product Reviews Transformation") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

df = spark.read.schema(review_schema).parquet(file)

print(df.count())
print(df.head(5))

3015910
[Row(reviewerID='AKHCHD39R5KKD', asin='B01BLI4FWI', reviewText="It's a really good case.  If it had the built in screen protector it would be a great case,  and would've received the other two stars."), Row(reviewerID='A299CPUUY3E1QF', asin='B00U2STI12', reviewText='When using the metal ring on the edge of the case it is almost impossible to charge the phone unless the ring is removed.'), Row(reviewerID='AY3G1U2SGFINR', asin='B004WKH46W', reviewText='it is correctly fit for HTC inspire .I got it for very less price and i am very happy with this'), Row(reviewerID='A1S7PR7750I9T6', asin='B00LP0KTEG', reviewText='Perfect fit. Installed myself . made my phone look like new. No move of those plastic proctors for me.'), Row(reviewerID='A6MTZJ8TS84S5', asin='B00QZRYLBQ', reviewText='Don\'t fall for "edge-to-edge coverage for maximum protection" description. It DOESN\'T cover whole iPhone 6s plus screen! Also, it is a flat cover. It didn\'t curve to fit the curved screen on iPhone 6s p

# Create data training table

In [29]:
BASE_FILE = "data/silver/reviews_Cell_Phones_and_Accessories_70.parquet"
DEST_FILE = "data/gold/training_data"

# Initialize Schema for the data
review_schema = StructType([
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("overall", DoubleType(), True),
])

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Amazon Product Reviews Transformation") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

df = spark.read.schema(review_schema).parquet(BASE_FILE)

# Drop the columns that are not needed (keep reviewerName for now)
df = df.drop("reviewerName", "reviewerID", "asin")

print(df.count())

# Create a new column for semantic detection
df = df.withColumn(
    "semanticDetect", 
    when(col("overall") > 3, "positive")
    .when(col("overall") < 3, "negative")
    .otherwise("neutral"))

# Drop the overall column
df = df.drop("overall")

print(df.show(5))

# Overwrite the training data table
df.write.mode("overwrite").parquet(DEST_FILE)
print(f"Training data table written to: {DEST_FILE}")

# Clear everything before stopping
spark.catalog.clearCache()

spark.stop()

7037123
+--------------------+--------------+
|          reviewText|semanticDetect|
+--------------------+--------------+
|great quality and...|      positive|
|The case is nice ...|      positive|
|Doesn't fit my ph...|      negative|
|Haven't used it a...|      positive|
|I love these cove...|      positive|
+--------------------+--------------+
only showing top 5 rows

None


Training data table written to: data/gold/training_data


In [30]:
con = connect()

# File 1: 70% of the data
file = "data/gold/training_data.parquet"

con.execute(f"""
    COPY (
        SELECT *
        FROM read_parquet('data/gold/training_data/*.parquet')
    )
    TO '{file}'
    (FORMAT PARQUET, COMPRESSION SNAPPY);
""")
print(f"✅ Wrote all records to {file}")


✅ Wrote all records to data/gold/training_data.parquet


In [32]:
# Initialize Schema for the data
file = "data/gold/training_data.parquet"

review_schema = StructType([
    StructField("reviewText", StringType(), True),
    StructField("semanticDetect", StringType(), True),
    
])

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Amazon Product Reviews Transformation") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

df = spark.read.schema(review_schema).parquet(file)

print(df.count())
print(df.head(5))

# Count sentiment distribution
df.groupBy("semanticDetect").count().show()



7037123
[Row(reviewText='great quality and nicely finished', semanticDetect='positive'), Row(reviewText='The case is nice really skinny fix good, but the screen protector does not stick in the phone, in the corners there are bubbles, so my recommendation is to purchase other good screen protector and place it as IQ shield, I did it and now it is good.', semanticDetect='positive'), Row(reviewText="Doesn't fit my phone. Auxiliary does not lign up. Cannot use this case", semanticDetect='negative'), Row(reviewText="Haven't used it a ton yet, but so far I am very happy. Not the highest quality, but definitely what I expected and its great for the price! I want more of these things because it really is very versatile and has many uses!", semanticDetect='positive'), Row(reviewText='I love these covers easy to put on the phone', semanticDetect='positive')]
+--------------+-------+
|semanticDetect|  count|
+--------------+-------+
|      positive|5010181|
|       neutral| 602391|
|      negativ